## 1. Environment Setup

In [ ]:
# Import NeqSim - Direct Java Access via jneqsim
from neqsim import jneqsim
import jpype
from jpype.types import JArray, JDouble
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import Java classes through the jneqsim gateway
SystemSrkEos = jneqsim.thermo.system.SystemSrkEos
Stream = jneqsim.process.equipment.stream.Stream
PipeFlowNetwork = jneqsim.process.equipment.network.PipeFlowNetwork
ProcessSystem = jneqsim.process.processmodel.ProcessSystem
AdvectionScheme = jneqsim.fluidmechanics.flowsolver.AdvectionScheme

plt.style.use('seaborn-v0_8-darkgrid')

## 2. Create Feed Fluids

We create two different gas compositions to demonstrate mixing at manifolds.

In [ ]:
def create_lean_gas(T_K=288.15, P_bar=100.0, flow_MSm3_day=5.0):
    """Create a lean natural gas (high methane content)."""
    gas = SystemSrkEos(T_K, P_bar)
    gas.addComponent('nitrogen', 0.02)
    gas.addComponent('CO2', 0.01)
    gas.addComponent('methane', 0.88)
    gas.addComponent('ethane', 0.05)
    gas.addComponent('propane', 0.03)
    gas.addComponent('n-butane', 0.01)
    gas.createDatabase(True)
    gas.setMixingRule('classic')
    gas.init(0)
    gas.init(3)
    gas.initPhysicalProperties()
    gas.setTotalFlowRate(flow_MSm3_day, 'MSm3/day')
    return gas

def create_rich_gas(T_K=290.15, P_bar=95.0, flow_MSm3_day=3.0):
    """Create a rich natural gas (higher C2+ content)."""
    gas = SystemSrkEos(T_K, P_bar)
    gas.addComponent('nitrogen', 0.01)
    gas.addComponent('CO2', 0.02)
    gas.addComponent('methane', 0.72)
    gas.addComponent('ethane', 0.11)
    gas.addComponent('propane', 0.08)
    gas.addComponent('n-butane', 0.06)
    gas.createDatabase(True)
    gas.setMixingRule('classic')
    gas.init(0)
    gas.init(3)
    gas.initPhysicalProperties()
    gas.setTotalFlowRate(flow_MSm3_day, 'MSm3/day')
    return gas

# Create feed streams
feed1 = Stream('feed 1', create_lean_gas())
feed2 = Stream('feed 2', create_rich_gas())
feed3 = Stream('feed 3', create_lean_gas(T_K=285.15, P_bar=98.0, flow_MSm3_day=4.0))

print('Feed streams created:')
print(f'  Feed 1: Lean gas, {feed1.getThermoSystem().getTotalNumberOfMoles():.0f} mol/s')
print(f'  Feed 2: Rich gas, {feed2.getThermoSystem().getTotalNumberOfMoles():.0f} mol/s')
print(f'  Feed 3: Lean gas, {feed3.getThermoSystem().getTotalNumberOfMoles():.0f} mol/s')

## 3. Build Pipeline Network

We create a network with:
- Two satellite manifolds (A and B) receiving feeds
- A central manifold collecting flows from both satellites
- An export pipeline from the central manifold

In [ ]:
# Create the network
network = PipeFlowNetwork('gathering network')

# Configure default parameters
network.setDefaultWallRoughness(1e-5)  # m
network.setDefaultOuterTemperature(278.0)  # K (ambient)
network.setDefaultHeatTransferCoefficients(5.0, 15.0)  # outer, wall [W/m2K]

# Enable compositional tracking with TVD scheme for sharp fronts
network.setCompositionalTracking(True)
network.setAdvectionScheme(AdvectionScheme.TVD_VAN_LEER)

# Create manifolds
manifold_A = network.createManifold('manifold A')
manifold_B = network.createManifold('manifold B')
central_manifold = network.createManifold('central manifold')

# Add inlet pipelines to manifold A
network.addInletPipeline('pipe 1', feed1, manifold_A, 
                         length=5000.0, diameter=0.3, numberOfNodes=25)
network.addInletPipeline('pipe 2', feed2, manifold_A, 
                         length=4500.0, diameter=0.25, numberOfNodes=22)

# Add inlet pipeline to manifold B
network.addInletPipeline('pipe 3', feed3, manifold_B, 
                         length=6000.0, diameter=0.28, numberOfNodes=30)

# Connect satellite manifolds to central manifold
network.connectManifolds(manifold_A, central_manifold, 'pipe A-Central',
                         length=8000.0, diameter=0.35, numberOfNodes=40)
network.connectManifolds(manifold_B, central_manifold, 'pipe B-Central',
                         length=7000.0, diameter=0.32, numberOfNodes=35)

print('Network topology:')
print(f'  Manifolds: {len(network.getManifolds())}')
print(f'  Pipelines: {len(network.getPipelines())}')
for seg in network.getPipelines():
    src = seg.getFromManifold() if seg.getFromManifold() else 'feed'
    print(f'    {seg.getName()}: {src} -> {seg.getToManifold()}')

## 4. Run Steady-State Simulation

In [ ]:
# Create process system and run
process = ProcessSystem()
process.add(feed1)
process.add(feed2)
process.add(feed3)
process.add(network)
process.run()

# Get outlet stream
outlet = network.getOutletStream()
outlet_system = outlet.getThermoSystem()

print('\nSteady-State Results:')
print(f'  Outlet flow rate: {outlet.getFlowRate("MSm3/day"):.2f} MSm3/day')
print(f'  Outlet pressure: {outlet_system.getPressure():.1f} bara')
print(f'  Outlet temperature: {outlet_system.getTemperature() - 273.15:.1f} °C')
print('\n  Outlet composition (mol%):')
for i in range(outlet_system.getNumberOfComponents()):
    comp = outlet_system.getComponent(i)
    print(f'    {comp.getComponentName()}: {comp.getz() * 100:.2f}%')

## 5. Extract and Plot Profiles

In [ ]:
# Get profiles for the export pipeline
pressure_profile = network.getPressureProfile('pipe A-Central', 'bara')
temp_profile = network.getTemperatureProfile('pipe A-Central', 'C')
methane_profile = network.getCompositionProfile('pipe A-Central', 'methane')

# Create distance array
n_nodes = len(pressure_profile)
distance_km = np.linspace(0, 8.0, n_nodes)

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Pressure profile
axes[0].plot(distance_km, pressure_profile, 'b-', linewidth=2)
axes[0].set_xlabel('Distance (km)')
axes[0].set_ylabel('Pressure (bara)')
axes[0].set_title('Pressure Profile: Pipe A-Central')
axes[0].grid(True)

# Temperature profile
axes[1].plot(distance_km, temp_profile, 'r-', linewidth=2)
axes[1].set_xlabel('Distance (km)')
axes[1].set_ylabel('Temperature (°C)')
axes[1].set_title('Temperature Profile: Pipe A-Central')
axes[1].grid(True)

# Composition profile
axes[2].plot(distance_km, np.array(methane_profile) * 100, 'g-', linewidth=2)
axes[2].set_xlabel('Distance (km)')
axes[2].set_ylabel('Methane (mass %)')
axes[2].set_title('Composition Profile: Pipe A-Central')
axes[2].grid(True)

plt.tight_layout()
plt.show()

## 6. Transient Simulation

Now let's run a transient simulation where we change the inlet composition at feed 1.

In [ ]:
import java.util.UUID as UUID

# Reset simulation time
network.resetSimulationTime()

# Run transient simulation for 10 minutes
dt = 60.0  # 60 second time steps
total_time = 600.0  # 10 minutes
n_steps = int(total_time / dt)

# Store results
time_history = []
outlet_pressure_history = []
outlet_methane_history = []

calc_id = UUID.randomUUID()

for step in range(n_steps):
    # Run transient step
    network.runTransient(dt, calc_id)
    
    # Record results
    time_history.append(network.getSimulationTime())
    outlet = network.getOutletStream()
    outlet_pressure_history.append(outlet.getThermoSystem().getPressure())
    methane_frac = outlet.getThermoSystem().getComponent('methane').getz()
    outlet_methane_history.append(methane_frac * 100)

print(f'Transient simulation complete: {n_steps} steps, {total_time/60:.0f} minutes')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

time_min = np.array(time_history) / 60

axes[0].plot(time_min, outlet_pressure_history, 'b-', linewidth=2)
axes[0].set_xlabel('Time (minutes)')
axes[0].set_ylabel('Outlet Pressure (bara)')
axes[0].set_title('Transient Outlet Pressure')
axes[0].grid(True)

axes[1].plot(time_min, outlet_methane_history, 'g-', linewidth=2)
axes[1].set_xlabel('Time (minutes)')
axes[1].set_ylabel('Methane (mol%)')
axes[1].set_title('Transient Outlet Methane Content')
axes[1].grid(True)

plt.tight_layout()
plt.show()

## 7. Summary

The `PipeFlowNetwork` class provides:

| Feature | Description |
|---------|-------------|
| Manifold modeling | Mixers that combine multiple incoming flows |
| Compositional tracking | Full component tracking through the network |
| Steady-state solver | TDMA-based solver with momentum, energy, mass balance |
| Transient solver | Dynamic simulations with time-stepping |
| TVD advection schemes | Sharp front tracking for composition changes |
| Heat transfer | Configurable ambient conditions and heat transfer |

This is suitable for:
- Gas gathering networks
- Pipeline networks with multiple feeds
- Gas switching studies (composition transitions)
- Transient pressure and flow analysis